## Finding Methane leakages using TROPOMI data.

The code that tries to detect methane leakages is separated into two parts, called the producer and the consumer. They communicate with each other using kafka, a data stream management system. The producer takes an input, a selection of days and regions to look at, downloads the corresponding TROPOMI data and outputs the path at which the files are stored. The consumer receives the paths, extracts features and detects leakages using anomaly detection.

### TROPOMI data


Our group has worked on the L2 CH4 data. The reason to choose this data is because L2 CH4 consists of methane data exclusively when compared to that of L1 data which consists of data of various gases. The biggest influence for annual methane changes in China are coal mines (H.Yang, 2020). Therefore, the biggest coal mines in China have been found, and this area has been chosen for the analysis of methane. The main reason is that bigger methane concentrations are easier to identify using TROPOMI data.


### Features

When thinking about the features that could be used to detect leakages, our group first thought about using object detection techniques. However the nature of the data prohibits the use of these techniques. The data is not grid like we are used to from pictures where the distance in the real world is mapped evenly. In the TROPOMI dataset, the pixels are not evenly spaced out. In other words, the delta in longitude and latitude between two pixels is not a constant. Because of this, our group decided not to pursue extracting spatial characteristics. Instead, we only use temporal characteristics.

Because using only one pixel only gives us concentrations, we decided to track locations over time. We want to use features that tell us the concentration and the change in concentration both absolute(delta_ch4) and relative(ratio). To increase robustness we compare two time frames rather than two points in time.

When requesting TROPOMI data for an area and timeframe, the API returns all the images available that overlap with the selected area. Because the images sometimes overlap, first a function is used that combines the overlapping pixels. After the overlapping is done, we then compare between these two time frames to get the features. A visualization of three features set (methane concentration difference, concentration ratio and actual concentration) is shown below. From the plotted features we can tell that methane difference and ratio are highly correlated, we will try some anomaly detection techniques to find outliers. (and validate them?) 


<figure>
  <img src="./1.png" style="zoom:100%;" />
  <figcaption>Figure 1. 3D plot for three features</figcaption>
</figure>

<figure>
  <img src="./2.png" style="zoom:100%;" />
  <figcaption>Figure 2. Histograms</figcaption>
</figure>

### Choosing anomaly detection method
Varun Chandola, Arindam Banerjee, and Vipin Kumar. (2009) Anomaly Detection: A Survey https://dl.acm.org/doi/pdf/10.1145/1541880.1541882
#### Nature of input data
The input data is the features that we extracted. All the features are measures of concentration in time or at a point in time. There are multiple features, so the data is multivariate. The input data is point data; there is no relationship between one instance and another.
Because of the point based nature of the input data, contextual and collective anomaly detection is not applicable. We focus on point anomalies.

#### Data labels
Our team has searched for labeled methane leakages but there is no central database for that. Another option we tried is to take examples from the papers that found leakages. Often this data is not publicly available. Therefore we decided to focus on unsupervised anomaly detection. Using unsupervised anomaly detection, we have to make the assumption that anomalies are infrequent relative to the normal instances. We believe that this assumption is correct. If this assumption is not true then such techniques suffer from a high false alarm rate.

#### Output of Anomaly Detection
The two main output types of anomaly detection techniques are scores and labels. Scores are a measure of how anomalous the instance is considered. Labels are binary and tell you whether an instance is anomalous or not. Scores as output would be preferred but this is not a deal-breaker.

### Techniques

#### Classification-based
Classification-based anomaly detection techniques are not applicable because they only work on labeled data.

#### Nearest neighbor-based
Nearest neighbor-based anomaly detection techniques work on non-labeled data. They make the assumption that normal data instances occur in dense neighborhoods, while anomalies occur far from their closest neighbors.
These methods generally calculate an anomaly score using the distance to kth nearest neighbor, or using relative density.
For the distance-based nearest neighbour the anomaly score of a data instance is defined as its distance to its kth nearest neighbor in a given data set. Instances with a large distance to their kth nearest neighbor are considered to be anomalies.
We used distance-based K nearest neighbours to make predictions to our feature set, and visualize the outliers using a 3D plot. As can be seen in the plot, two points who have a large distance to their neighbours are considered outliers.


<figure>
  <img src="./3.png" style="zoom:100%;" />
  <figcaption>Figure 3. Distance based K nearest Neighbour</figcaption>
</figure>

Density-based anomaly detection techniques calculates the density of the neighborhood
of each data instance. Instances that fall into a low density neighbourhood are considered to be anomalous, while instances in neighbourhoods with a high density are considered to be normal. A plot of applying density-based K nearest neighbor on our data is shown below.


<figure>
  <img src="./4.png" style="zoom:100%;" />
  <figcaption>Figure 4. Distance based K nearest Neighbour</figcaption>
</figure>

The main disadvantage of nearest neighbor-based anomaly detection techniques is the bad computational performance.
Nearest neighbor-based anomaly detection techniques are a good fit if we can confirm that our data meets the assumption.


### Clustering-based


Clustering-based anomaly detection techniques work on non-labeled data. These techniques are categorized by the assumptions made on the input-data.
The first group makes the following assumption: normal data instances belong to a cluster in the data, while anomalies do not belong to any cluster. This requires a clustering algorithm that does not force instances into a cluster. As shown before, the outliers in our data set can form their own cluster, hence, our data doesn’t fit this assumption.

The second group makes the following assumption: normal data instances lie close to their closest cluster centroid, while anomalies are far away from their closest cluster centroid. K-Means clustering can be used for data under this assumption. We don’t believe this assumption is right.

The third group makes the following assumption: normal data instances belong to large and dense clusters, while anomalies either belong to small or sparse clusters. Techniques based on this assumption declare instances belonging to clusters whose size and/or density is below a threshold, as anomalous. A typical technique based on this assumption is CBLOF (Cluster-Based Local Outlier Factor), this technique will assign an anomaly score to each data instance, the score represents the size of the cluster to which the data instance belongs, as well as the distance of the data instance to its cluster centroid. And if the size and/or distance is below a threshold, the data instance is considered an anomaly. 
Another technique is DBSCAN (Density-based spatial clustering of applications with noise), it views clusters as areas of high density separated by areas of low density.
The computational performance is often bad for clustering-based techniques.


<figure>
  <img src="./5.png" style="zoom:100%;" />
  <figcaption>Figure 5. DBSCAN</figcaption>
</figure>

<figure>
  <img src="./6.png" style="zoom:100%;" />
  <figcaption>Figure 6. CBLOF</figcaption>
</figure>

#### Statistical-based
Statistical-based anomaly detection techniques rely on the following key assumption: Normal data instances occur in high probability regions of a stochastic model, while anomalies occur in the low probability regions of the stochastic model.
Within statistical-based anomaly detection, based on whether or not the model structure is pre-defined or determined from given data, it can be categorized into parametric and nonparametric techniques.
#### Spectral-based
Spectral techniques try to find an approximation of the data using a combination of attributes that capture the bulk of the variability in the data. Such techniques are based on the following key assumption: data can be embedded into a lower dimensional subspace in which normal instances and anomalies appear significantly different. It’s not clear to us  if this assumption can be satisfied and therefore decided not to pursue these techniques.
#### Chosen technique
Based on our assumption of our data set that anomaly instances are infrequent and occur far from their closest neighbors, the density-based clustering and distance-based K nearest neighbor method are chosen for anomaly detection.
